# CIFAR-100 Uppgift - CNN, KerasTuner och Transfer Learning

In [ ]:
# Importera bibliotek
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.callbacks import EarlyStopping

import keras_tuner as kt

## 1. Ladda och Förbereda Data

In [ ]:
# Ladda CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Visa dimensioner
print(f"Training data shape: {x_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Test data shape: {x_test.shape}")
print(f"Test labels shape: {y_test.shape}")
print(f"Number of classes: {len(np.unique(y_train))}")

In [ ]:
# Normalisera pixelvärdena till [0, 1]
x_train_normalized = x_train.astype('float32') / 255.0
x_test_normalized = x_test.astype('float32') / 255.0

# One-hot encoding av labels
y_train_categorical = to_categorical(y_train, 100)
y_test_categorical = to_categorical(y_test, 100)

print(f"Normalized training data shape: {x_train_normalized.shape}")
print(f"One-hot encoded labels shape: {y_train_categorical.shape}")

In [ ]:
# Visualisera några exempel från datasetet
plt.figure(figsize=(12, 12))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(x_train[i])
    plt.title(f"Class: {y_train[i][0]}")
    plt.axis('off')
plt.tight_layout()
plt.show()

## 2. Del a) Baseline CNN-modell

Skapa en grundläggande CNN-modell för att prediktera CIFAR-100 datasetet.

In [ ]:
def create_baseline_cnn():
    """Skapa en baseline CNN-modell"""
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        
        Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        
        Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        
        Flatten(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(100, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Skapa modellen
baseline_model = create_baseline_cnn()
baseline_model.summary()

In [ ]:
# Träna baseline-modellen
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history_baseline = baseline_model.fit(
    x_train_normalized, y_train_categorical,
    epochs=30,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

In [ ]:
# Utvärdera baseline-modellen
baseline_loss, baseline_accuracy = baseline_model.evaluate(x_test_normalized, y_test_categorical, verbose=0)
print(f"\n=== BASELINE CNN RESULTAT ===")
print(f"Test Accuracy: {baseline_accuracy:.4f}")
print(f"Test Loss: {baseline_loss:.4f}")

In [ ]:
# Visualisera träningshistorik för baseline
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history_baseline.history['accuracy'], label='Training Accuracy')
plt.plot(history_baseline.history['val_accuracy'], label='Validation Accuracy')
plt.title('Baseline CNN - Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history_baseline.history['loss'], label='Training Loss')
plt.plot(history_baseline.history['val_loss'], label='Validation Loss')
plt.title('Baseline CNN - Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 3. Del b) Hyperparameter-tuning med KerasTuner

Använd KerasTuner för att hitta optimala hyperparametrar.

In [ ]:
def build_tuner_model(hp):
    """Bygg modell med tuningsbara hyperparametrar"""
    model = Sequential()
    
    # Första Conv-layer
    model.add(Conv2D(
        filters=hp.Int('conv_1_filters', min_value=32, max_value=128, step=32),
        kernel_size=(3, 3),
        padding='same',
        activation='relu',
        input_shape=(32, 32, 3)
    ))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Andra Conv-layer
    model.add(Conv2D(
        filters=hp.Int('conv_2_filters', min_value=64, max_value=256, step=64),
        kernel_size=(3, 3),
        padding='same',
        activation='relu'
    ))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Tredje Conv-layer
    model.add(Conv2D(
        filters=hp.Int('conv_3_filters', min_value=128, max_value=512, step=128),
        kernel_size=(3, 3),
        padding='same',
        activation='relu'
    ))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    
    # Dropout rate
    model.add(Dropout(hp.Float('dropout_1', min_value=0.3, max_value=0.7, step=0.1)))
    
    # Dense layer
    model.add(Dense(
        units=hp.Int('dense_units', min_value=128, max_value=512, step=128),
        activation='relu'
    ))
    model.add(Dropout(hp.Float('dropout_2', min_value=0.3, max_value=0.7, step=0.1)))
    
    model.add(Dense(100, activation='softmax'))
    
    # Learning rate tuning
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
# Skapa en tuner (RandomSearch)
tuner = kt.RandomSearch(
    build_tuner_model,
    objective='val_accuracy',
    max_trials=10,  # Öka för bättre resultat (tar längre tid)
    executions_per_trial=1,
    directory='keras_tuner_dir',
    project_name='cifar100_tuning'
)

print(tuner.search_space_summary())

In [ ]:
# Kör sökningen efter bästa hyperparametrar
early_stopping_tuner = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

tuner.search(
    x_train_normalized, y_train_categorical,
    epochs=20,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stopping_tuner],
    verbose=1
)

In [ ]:
# Hämta bästa modellen
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

print("\n=== BÄSTA HYPERPARAMETRAR ===")
print(f"Conv Layer 1 Filters: {best_hyperparameters.get('conv_1_filters')}")
print(f"Conv Layer 2 Filters: {best_hyperparameters.get('conv_2_filters')}")
print(f"Conv Layer 3 Filters: {best_hyperparameters.get('conv_3_filters')}")
print(f"Dense Units: {best_hyperparameters.get('dense_units')}")
print(f"Dropout 1: {best_hyperparameters.get('dropout_1')}")
print(f"Dropout 2: {best_hyperparameters.get('dropout_2')}")
print(f"Learning Rate: {best_hyperparameters.get('learning_rate')}")

In [ ]:
# Utvärdera den tunade modellen
tuned_loss, tuned_accuracy = best_model.evaluate(x_test_normalized, y_test_categorical, verbose=0)
print(f"\n=== KERASTUNER RESULTAT ===")
print(f"Test Accuracy: {tuned_accuracy:.4f}")
print(f"Test Loss: {tuned_loss:.4f}")
print(f"\nFörbättring jämfört med baseline: {(tuned_accuracy - baseline_accuracy):.4f} ({(tuned_accuracy - baseline_accuracy)/baseline_accuracy*100:.2f}%)")

## 4. Del c) Transfer Learning

Använd en förtränad modell (t.ex. VGG16 eller ResNet50) för att förbättra resultaten.

In [ ]:
# Förbereda data för transfer learning (vissa modeller kräver större bilder)
# Vi behåller 32x32 för CIFAR-100, men vissa modeller fungerar bättre med större storlekar

def create_transfer_learning_model(base_model_name='VGG16'):
    """Skapa transfer learning modell"""
    
    # Ladda förtränad modell utan top layers
    if base_model_name == 'VGG16':
        base_model = VGG16(
            weights='imagenet',
            include_top=False,
            input_shape=(32, 32, 3)
        )
    elif base_model_name == 'ResNet50':
        base_model = ResNet50(
            weights='imagenet',
            include_top=False,
            input_shape=(32, 32, 3)
        )
    
    # Frys alla lager i base model
    base_model.trainable = False
    
    # Bygg den kompletta modellen
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(100, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Skapa transfer learning modell
transfer_model = create_transfer_learning_model('VGG16')
transfer_model.summary()

In [ ]:
# Träna transfer learning modellen
early_stopping_transfer = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history_transfer = transfer_model.fit(
    x_train_normalized, y_train_categorical,
    epochs=30,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stopping_transfer],
    verbose=1
)

In [ ]:
# Utvärdera transfer learning modellen
transfer_loss, transfer_accuracy = transfer_model.evaluate(x_test_normalized, y_test_categorical, verbose=0)
print(f"\n=== TRANSFER LEARNING RESULTAT ===")
print(f"Test Accuracy: {transfer_accuracy:.4f}")
print(f"Test Loss: {transfer_loss:.4f}")
print(f"\nFörbättring jämfört med baseline: {(transfer_accuracy - baseline_accuracy):.4f} ({(transfer_accuracy - baseline_accuracy)/baseline_accuracy*100:.2f}%)")

In [ ]:
# Visualisera träningshistorik för transfer learning
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history_transfer.history['accuracy'], label='Training Accuracy')
plt.plot(history_transfer.history['val_accuracy'], label='Validation Accuracy')
plt.title('Transfer Learning - Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history_transfer.history['loss'], label='Training Loss')
plt.plot(history_transfer.history['val_loss'], label='Validation Loss')
plt.title('Transfer Learning - Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 5. Jämförelse av Resultat

In [ ]:
# Sammanställning av resultat
results = {
    'Metod': ['Baseline CNN', 'KerasTuner', 'Transfer Learning'],
    'Test Accuracy': [baseline_accuracy, tuned_accuracy, transfer_accuracy],
    'Test Loss': [baseline_loss, tuned_loss, transfer_loss]
}

import pandas as pd
results_df = pd.DataFrame(results)
results_df['Accuracy (%)'] = results_df['Test Accuracy'] * 100
print("\n=== SAMMANSTÄLLNING AV RESULTAT ===")
print(results_df.to_string(index=False))
print(f"\nBästa metod: {results_df.loc[results_df['Test Accuracy'].idxmax(), 'Metod']}")
print(f"Högsta Accuracy: {results_df['Test Accuracy'].max():.4f}")

In [ ]:
# Visualisera jämförelse
plt.figure(figsize=(10, 6))
methods = ['Baseline CNN', 'KerasTuner', 'Transfer Learning']
accuracies = [baseline_accuracy, tuned_accuracy, transfer_accuracy]

bars = plt.bar(methods, accuracies, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
plt.ylabel('Test Accuracy')
plt.title('Jämförelse av Modell-prestanda på CIFAR-100')
plt.ylim([0, 1])

# Lägg till värden på toppen av varje stapel
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.4f}',
             ha='center', va='bottom')

plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Slutsatser
### Svar på uppgifterna:
**a) Baseline CNN-modell:**
- Skapade en grundläggande CNN med flera konvolutionella lager
- Modellen fungerar som utgångspunkt för jämförelse

**b) Hyperparameter-tuning med KerasTuner:**
- Använde RandomSearch för att hitta optimala hyperparametrar
- Justerade antal filter, dense units, dropout rates och learning rate
- Resultat: [Jämför med baseline för att se om det blev bättre]

**c) Transfer Learning:**
- Använde förtränad VGG16 från ImageNet
- Frös base model och tränade endast de nya lagren
- Resultat: [Jämför med baseline och KerasTuner]